<a href="https://colab.research.google.com/github/KimJae-hee/kyakyakyakya/blob/master/%ED%83%9C%EC%96%91%EA%B4%91_%EB%B0%9C%EC%A0%84_%EC%98%88%EC%B8%A1_0727.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
np.random.seed(0)

Using TensorFlow backend.


### 우선 One-Hot Encoding으로 진행하자.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
sun = pd.read_csv('/content/drive/My Drive/P-SAT/태양광 예측/진짜 데이터 셋/final_data0723_3.csv', encoding = 'CP949')
sun["ymd"] = pd.to_datetime(sun["ymd"], format = "%Y-%m-%d")
sun = sun.set_index(pd.to_datetime(sun["date"]))
del sun["date"]
del sun["day"]
del sun["wday"]

In [4]:
sun_test = sun[sun["y"].isnull()]
sun_train = sun[sun["y"].notnull()]

In [5]:
y_train = sun_train["y"]
y_test = sun_test["y"]
# 각각의 인코딩을 위해 데이터 프레임 복제
sun_mne = sun_train.drop(["ymd"], axis = 1)
sun_ohe = sun_train.drop(["y", "ymd"], axis = 1)
sun_tets = sun_test.drop(["y"], axis = 1)

### train, validation, test 인덱스 추출 및 데이터 분리

In [7]:
idx_train_1 = (sun_train["ymd"] < "2020-01-24")
idx_train_2 = ((sun_train["ymd"] < "2020-03-24"))
idx_train_3 = ((sun_train["ymd"] < "2020-05-24"))

idx_val_1 = ((sun_train["ymd"] > "2020-01-23") & (sun_train["ymd"] <"2020-01-31"))
idx_val_2 = ((sun_train["ymd"] > "2020-03-23") & (sun_train["ymd"] <"2020-03-31"))
idx_val_3 = ((sun_train["ymd"] > "2020-05-23") & (sun_train["ymd"] <"2020-05-31"))

idx_test_1 = (sun_test["ymd"] == "2020-01-31")
idx_test_2 = (sun_test["ymd"] == "2020-03-31")
idx_test_3 = (sun_test["ymd"] == "2020-05-31")

In [9]:
x_train_1 = sun_ohe[idx_train_1]
x_train_2 = sun_ohe[idx_train_2]
x_train_3 = sun_ohe[idx_train_3]

x_val_1 = sun_ohe[idx_val_1]
x_val_2 = sun_ohe[idx_val_2]
x_val_3 = sun_ohe[idx_val_3]

x_test_1 = sun_test[idx_test_1]
x_test_2 = sun_test[idx_test_2]
x_test_3 = sun_test[idx_test_3]

y_train_1 = y_train[idx_train_1]
y_train_2 = y_train[idx_train_2]
y_train_3 = y_train[idx_train_3]

y_val_1 = y_train[idx_val_1]
y_val_2 = y_train[idx_val_2]
y_val_3 = y_train[idx_val_3]

y_test_1 = y_test[idx_test_1]
y_test_2 = y_test[idx_test_2]
y_test_3 = y_test[idx_test_3]



print(y_train_1.isnull().sum())
print(y_train_2.isnull().sum())
print(y_train_3.isnull().sum())
print(y_val_1.isnull().sum())
print(y_val_2.isnull().sum())
print(y_val_3.isnull().sum())

0
0
0
0
0
0


### One-Hot-Encoder 및 Ordianl-Encoder

In [10]:
cate_list = ["month", "rain_type", "skycondition", "pm10_cat", "pm2_5_cat", "rain_yn", "hour" ] #인코딩을 위한 변수 
sun_ohe_columns = sun_ohe.columns.values.tolist()

In [11]:
non_cate_list = sun_ohe_columns
for i in range(len(cate_list)):
  non_cate_list.remove(cate_list[i])

In [30]:
def onehotencoder_minmax(x_train, x_val):
  shape_x_train = x_train.shape
  shape_x_val = x_val.shape
  
  print(f"인코딩 전 트레인 데이터의 shape은 {shape_x_train}입니다.")
  print(f"인코딩 전 평가 데이터의 shape은 {shape_x_val}입니다.")

  x_train_cat = x_train[cate_list]
  x_train_noncat = x_train[non_cate_list]

  x_val_cat = x_val[cate_list]
  x_val_noncat = x_val[non_cate_list]

  enc = OneHotEncoder(handle_unknown='error')
  enc.fit(x_train_3[cate_list])
  x_train_ohe = enc.transform(x_train_cat).toarray()
  x_val_ohe = enc.transform(x_val_cat).toarray()

  x_train_noncat = np.asarray(x_train_noncat)
  x_val_noncat = np.asarray(x_val_noncat)

  scaler = RobustScaler()
  scaler.__init__
  scaler.fit(x_train_noncat)
  x_train_noncat = scaler.transform(x_train_noncat)
  x_val_noncat = scaler.transform(x_val_noncat)

  x_train_result = np.hstack((x_train_noncat, x_train_ohe))
  x_val_result = np.hstack((x_val_noncat, x_val_ohe))

  shape_x_train_done = x_train_result.shape
  shape_x_val_done = x_val_result.shape
  print(f"인코딩 후 트레인 데이터의 shape은 {shape_x_train_done}입니다.")
  print(f"인코딩 후 평가 데이터의 shape은 {shape_x_val_done}입니다.")
  print("   ")
  return x_train_result, x_val_result                   

In [32]:
x_train_1_ohe, x_val_1_ohe = onehotencoder_minmax(x_train_1, x_val_1)
x_train_2_ohe, x_val_2_ohe = onehotencoder_minmax(x_train_2, x_val_2)
x_train_3_ohe, x_val_3_ohe = onehotencoder_minmax(x_train_3, x_val_3)

인코딩 전 트레인 데이터의 shape은 (4944, 28)입니다.
인코딩 전 평가 데이터의 shape은 (168, 28)입니다.
인코딩 후 트레인 데이터의 shape은 (4944, 73)입니다.
인코딩 후 평가 데이터의 shape은 (168, 73)입니다.
   
인코딩 전 트레인 데이터의 shape은 (6360, 28)입니다.
인코딩 전 평가 데이터의 shape은 (168, 28)입니다.
인코딩 후 트레인 데이터의 shape은 (6360, 73)입니다.
인코딩 후 평가 데이터의 shape은 (168, 73)입니다.
   
인코딩 전 트레인 데이터의 shape은 (7800, 28)입니다.
인코딩 전 평가 데이터의 shape은 (168, 28)입니다.
인코딩 후 트레인 데이터의 shape은 (7800, 73)입니다.
인코딩 후 평가 데이터의 shape은 (168, 73)입니다.
   


### train, val, test set 생성

In [33]:
data_set_list = [[x_train_1_ohe, y_train_1, x_val_1_ohe, y_val_1],
                 [x_train_2_ohe, y_train_2, x_val_2_ohe, y_val_2],
                 [x_train_3_ohe, y_train_3, x_val_3_ohe, y_val_3]]

### 손실함수 정의

In [34]:
def lossloss(y, y_hat):
  result = []
  for i in range(int(y.shape[0]/24)):
    start = 24*i
    end = (24*(i+1))
    tmp = sum(abs(y[start:end] - y_hat[start:end])*y[start:end]/sum(y[start:end]))/133
    result.append(tmp)
  loss = np.mean(result)
  return loss

def RMSE(y, y_hat):
  result = mse(y, y_hat)**0.5
  return result

### 일반화 코드

### 모델 정의

In [48]:
dim_x_train_1 = x_train_1_ohe.shape
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
# 2. 모델 구성하기
model = Sequential()
model.__init__
model.add(Dense(1, input_dim= dim_x_train_1[1], activation='relu'))
model.add(Dense(70, activation='relu'))
model.add(Dense(1))
# 3. 모델 학습과정 설정하기
model.compile(optimizer = "rmsprop", loss = root_mean_squared_error, 
              metrics =["mse"])

# 4. 모델 학습시키기
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

### 1월, 3월, 5월에 맞게 데이터 학습 후 예측 및 오차 계산

In [39]:
# 모델링 함수 
def modeling(data_set, model, batchsize = 64, epoch_n = 300):
  weighted = []
  rmse = []
  y_hat_result = []
  for i in range(len(data_set)):
    X_train = data_set[i][0]
    Y_train = data_set[i][1]
    X_val = data_set[i][2]
    Y_val = data_set[i][3]

    model.fit(X_train, Y_train, epochs = epoch_n, batch_size = batchsize, validation_split = 0.3, verbose = 0, callbacks = [mc, early_stopping] )
    
    y_hat = model.predict(X_val)
    y_hat = y_hat.flatten()
    y_hat_tmp = y_hat

    tmp_weighted = lossloss(Y_val, y_hat)
    tmp_rmse = RMSE(Y_val, y_hat)

    weighted.append(tmp_weighted)
    rmse.append(tmp_rmse)
    y_hat_result = y_hat_tmp
    print(i, "번째 validiaion 결과는 다음과 같습니다.", ":::", "가중평균: ", tmp_weighted, "RMSE: ", tmp_rmse )

  print("   ")
 # return y_hat_result
  return weighted

In [49]:
modeling(data_set_list, model)

0 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.22802001456565427 RMSE:  21.34854366290836
Epoch 00101: early stopping
1 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.3964479068136485 RMSE:  34.50438031156971
Epoch 00101: early stopping
2 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.37288972892045 RMSE:  34.40925789795661


array([15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296585,
       15.3296585, 15.3296585, 15.3296585, 15.3296585, 15.3296

In [37]:
dim_x_train_1 = x_train_1_ohe.shape
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
model_list = []
for i in range(0, 10):
  for j in range(0, 5):
    model_tmp = Sequential()
    model_tmp.__init__
    model_tmp.add(Dense(1, input_dim= dim_x_train_1[1], activation='relu'))
    model_tmp.add(Dense((i*6 + 10), activation='relu'))
    model_tmp.add(Dense((j*3 + 10), activation='relu'))
    model_tmp.add(Dense(1))
    # 3. 모델 학습과정 설정하기
    model_tmp.compile(optimizer = "rmsprop", loss = root_mean_squared_error, 
                metrics =["mse"])
    model_list.append(model_tmp)

In [80]:
x = model_list[1]

0

### 배치 사이즈 그리드 서치: 큰 성능 변화 

In [66]:
for i in range(0, 20):
  for j in [16, 32, 48, 80]:
    print(f"++++++++++++++++++++++batchsize: {j}, node_n: {(i*3) + 10} 모델 시작++++++++++++++++++++++")
    weighted_result = modeling(data_set_list, model_list[i], batchsize=j)
    print(f"모델 종료//// 모델 성능:{np.mean(weighted_result)} ")
    print(f"++++++++++++++++++++++batchsize: {j}, node_n: {i} 모델 끝++++++++++++++++++++++")
    print("   ") 
    print("###############################################################################################################")

++++++++++++++++++++++batchsize: 16, node_n: 10 모델 시작++++++++++++++++++++++
Epoch 00102: early stopping
0 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.08784632340325979 RMSE:  7.97460758662167
Epoch 00297: early stopping
1 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.11945223256302497 RMSE:  11.702414775076491
Epoch 00103: early stopping
2 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.11458283694160551 RMSE:  11.60378552472376
   
모델 종료//// 모델 성능:0.10729379763596343 
++++++++++++++++++++++batchsize: 16, node_n: 0 모델 시작++++++++++++++++++++++
   
++++++++++++++++++++++batchsize: 32, node_n: 10 모델 시작++++++++++++++++++++++
Epoch 00101: early stopping
0 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.09329432898579464 RMSE:  7.7593887947652105
Epoch 00158: early stopping
1 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.10884127542712842 RMSE:  11.184956358401381
Epoch 00219: early stopping
2 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.133296017960621 RMSE:  13.141055819133022
   
모델 종료//// 모델 성능:0.11181054079118136 
++

KeyboardInterrupt: ignored

### 레이어 2개짜리 모델 그리드 서치 

In [41]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
for i in range(len(model_list)):
  print(f"++++++++++++++++++++++{i}번 모델 시작++++++++++++++++++++++")
  weighted_result = modeling(data_set_list, model_list[i], batchsize=128)
  print(f"모델 종료//// 모델 성능:{np.mean(weighted_result)} ")
  print(f"+++++++++++++++++++++++{i}번 모델 끝++++++++++++++++++++++")
  print("   ") 
  print("###############################################################################################################")

++++++++++++++++++++++0번 모델 시작++++++++++++++++++++++
Epoch 00228: early stopping
0 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.09747379787017288 RMSE:  7.919084191918822
Epoch 00111: early stopping
1 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.11462032179322579 RMSE:  11.375894992628565
Epoch 00108: early stopping
2 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.08999483327724044 RMSE:  9.3970090487961
   
모델 종료//// 모델 성능:0.1006963176468797 
+++++++++++++++++++++++0번 모델 끝++++++++++++++++++++++
   
###############################################################################################################
++++++++++++++++++++++1번 모델 시작++++++++++++++++++++++
Epoch 00232: early stopping
0 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.08403395456376993 RMSE:  8.705998121791529
Epoch 00140: early stopping
1 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.12722148550374016 RMSE:  12.124443973072449
Epoch 00103: early stopping
2 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.10650379136911474 RMSE:  10.831412889115267


최고 성능: 노드 50개짜리

In [ ]:
for i in range(len(data_set_list)):
  X_train = data_set_list[i][0]
  Y_train = data_set_list[i][1]
  X_val = data_set_list[i][2]
  Y_val = data_set_list[i][3]
  print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

(4944, 57) (4944,) (168, 57) (168,)
(7800, 57) (6360,) (168, 57) (168,)
(6360, 57) (7800,) (168, 57) (168,)


#### 내일 할일: 
                1. 인코딩 방식 바꿔보기
                2. 모델 구조 바꿔보기 
                3. 1~2를 함수로 짜서 한번에 돌리기
